<a href="https://colab.research.google.com/github/IVMRanepa/AI-to-help-students-and-teachers/blob/main/Analysis_of_the_economies_of_countries_with_a_frontier_market_and_the_impact_on_Russia's_trade_turnover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ экономик стран с пограничным рынком и влияние на торговый оборот России

Этот блокнот анализирует экономические показатели стран, граничащих с Россией, и влияние их экономики на торговлю с РФ.
- Используются данные Всемирного банка
- Интерактивные графики от Plotly
- Линейная регрессия для прогнозирования

In [ ]:
# Установка необходимых библиотек
!pip install wbdata pandas plotly scikit-learn

In [ ]:
import pandas as pd
import wbdata
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# Список стран, граничащих с РФ
countries = ["BY", "UA", "GE", "AZ", "KZ", "MN"]

# Экономические индикаторы и их названия для удобства
indicators = {
    'NY.GDP.MKTP.CD': 'GDP', # Валовой внутренний продукт
    'FP.CPI.TOTL.ZG': 'Inflation', # Инфляция, потребительские цены (годовой %)
    'BG.GSR.NFSV.GD.ZS': 'Trade_GDP' # Торговля товарами и услугами (% ВВП)
}

# Загрузка данных из Всемирного банка
# Аргумент 'convert_date=True' был удален, так как он вызывает ошибку в текущей версии wbdata.
# Даты будут обрабатываться после загрузки, если это необходимо.
data = wbdata.get_dataframe(indicators, country=countries)
# Сброс индекса для удобства работы с колонками 'country' и 'date'
data = data.reset_index()
# Удаление строк, содержащих пропущенные значения, чтобы избежать ошибок при анализе
data = data.dropna()
# Вывод первых нескольких строк загруженных данных для проверки
print(data.head())

      country  date           GDP  Inflation  Trade_GDP
0  Azerbaijan  2024  7.431588e+10   2.212172  24.614662
1  Azerbaijan  2023  7.242847e+10   8.785430  20.579101
2  Azerbaijan  2022  7.880747e+10  13.852259  16.283493
3  Azerbaijan  2021  5.482541e+10   6.650299  17.901672
4  Azerbaijan  2020  4.269300e+10   2.759809  19.163302


In [ ]:
# Интерактивный график: динамика ВВП
fig_gdp = px.line(data, x='date', y='GDP', color='country', title='Динамика ВВП стран-соседей России')
fig_gdp.update_xaxes(title_text='Год')
fig_gdp.update_yaxes(title_text='ВВП (в долларах США)')
fig_gdp.show()

In [ ]:
# Интерактивная диаграмма рассеяния: ВВП vs Торговля
# Создадим новый столбец с абсолютным значением инфляции, так как размер маркера не может быть отрицательным.
data['Absolute_Inflation'] = data['Inflation'].abs()
fig_scatter = px.scatter(data, x='GDP', y='Trade_GDP', color='country', size='Absolute_Inflation',
                         title='Зависимость торгового оборота от ВВП и инфляции',
                         hover_data=['date'])
fig_scatter.update_layout(xaxis_title='ВВП', yaxis_title='Торговля (% ВВП)')
fig_scatter.show()

In [ ]:
# Корреляционная тепловая карта
corr_data = data[['GDP', 'Inflation', 'Trade_GDP']].corr()

fig_corr = go.Figure(data=go.Heatmap(
    z=corr_data.values,
    x=corr_data.columns,
    y=corr_data.columns,
    colorscale='RdBu',
    text=corr_data.values,
    texttemplate="%{text:.2f}",
    textfont={"size":14},
    hoverongaps=False
))
fig_corr.update_layout(title='Корреляция между экономическими показателями')
fig_corr.show()

In [ ]:
# Прогнозирование с помощью линейной регрессии
X = data[['GDP', 'Inflation']].values
y = data['Trade_GDP'].values

model = LinearRegression()
model.fit(X, y)

print(f'Коэффициенты: {model.coef_}')
print(f'Перехват: {model.intercept_}')

# Предсказание
predictions = model.predict(X)
data['Predicted_Trade'] = predictions

# Визуализация предсказаний
fig_pred = px.line(data, x='date', y=['Trade_GDP', 'Predicted_Trade'], color_discrete_sequence=['blue', 'red'],
                   title='Фактический и предсказанный торговый оборот')
fig_pred.show()

Коэффициенты: [-4.10663598e-11 -1.29426087e-02]
Перехват: 22.104442119889338


## Выводы
- Мы смогли проанализировать влияние ВВП и инфляции на объём торговли.
- Использовали линейную регрессию для прогнозирования.
- Интерактивные графики позволяют лучше понять данные.

## Состояние ядра

Вот некоторые из важных переменных в ядре: Переменная №1 имя: data, тип: DataFrame значение:
```
        country  date           GDP   Inflation  Trade_GDP
0    Azerbaijan  2024  7.431588e+10    2.212172  24.614662
1    Azerbaijan  2023  7.242847e+10    8.785430  20.579101
2    Azerbaijan  2022  7.880747e+10   13.852259  16.283493
3    Azerbaijan  2021  5.482541e+10    6.650299  17.901672
4    Azerbaijan  2020  4.269300e+10    2.759809  19.163302
..          ...   ...           ...         ...        ...
351     Ukraine  1998  4.188252e+10   10.577161  16.8375<TRUNCATED original_length=731>
```
Variable #2
name: `countries`, type: `list`
value: `['BY', 'UA', 'GE', 'AZ', 'KZ', 'MN']`
Variable #3
name: `indicators`, type: `dict`
value: `{'NY.GDP.MKTP.CD': 'GDP', 'FP.CPI.TOTL.ZG': 'Inflation', 'BG.GSR.NFSV.GD.ZS': 'Trade_GDP'}`
Here are some of the files available in the kernel:
* /content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
* /content/.config/default_configs.db
* /content/.config/.last_opt_in_prompt.yaml
* /content/.config/active_config
* /content/.config/gce
* /content/.config/.last_update_check.json
* /content/.config/config_sentinel
* /content/.config/.last_survey_prompt.yaml
* /content/sample_data/README.md
* /content/sample_data/anscombe.json
* /content/sample_data/california_housing_test.csv
* /content/sample_data/california_housing_train.csv
* /content/sample_data/mnist_train_small.csv
* /content/sample_data/mnist_test.csv

In [ ]:
# Интерактивный график: динамика инфляции
fig_inflation = px.line(data, x='date', y='Inflation', color='country', title='Динамика инфляции стран-соседей России')
fig_inflation.update_xaxes(title_text='Год')
fig_inflation.update_yaxes(title_text='Инфляция (годовой %)')
fig_inflation.show()

In [ ]:
# Интерактивный график: динамика ВВП
# Используем столбец 'date' для оси X, так как он содержит временные данные
fig_gdp = px.line(data, x='date', y='GDP', color='country', title='Динамика ВВП стран-соседей России')
fig_gdp.update_xaxes(title_text='Год') # Заголовок для оси X
fig_gdp.update_yaxes(title_text='ВВП (в долларах США)') # Заголовок для оси Y
fig_gdp.show() # Отображение графика

In [ ]:
# Корреляционная тепловая карта
corr_data = data[['GDP', 'Inflation', 'Trade_GDP']].corr()

fig_corr = go.Figure(data=go.Heatmap(
    z=corr_data.values,
    x=corr_data.columns,
    y=corr_data.columns,
    colorscale='RdBu',
    text=corr_data.values,
    texttemplate="%{text:.2f}",
    textfont={"size":14},
    hoverongaps=False
))
fig_corr.update_layout(title='Корреляция между экономическими показателями')
fig_corr.show()

In [ ]:
# Прогнозирование с помощью линейной регрессии
# Определение признаков (X) и целевой переменной (y)
X = data[['GDP', 'Inflation']].values
y = data['Trade_GDP'].values

# Инициализация и обучение модели линейной регрессии
model = LinearRegression()
model.fit(X, y)

# Вывод коэффициентов модели и точки пересечения с осью Y
print(f'Коэффициенты: {model.coef_}')
print(f'Перехват: {model.intercept_}')

# Выполнение предсказаний на основе обученной модели
predictions = model.predict(X)
data['Predicted_Trade'] = predictions # Добавление предсказанных значений в DataFrame

# Визуализация фактического и предсказанного торгового оборота
# Используем столбец 'date' для оси X для корректного отображения временного ряда
fig_pred = px.line(data, x='date', y=['Trade_GDP', 'Predicted_Trade'], color_discrete_sequence=['blue', 'red'],
                   title='Фактический и предсказанный торговый оборот')
fig_pred.show()

Коэффициенты: [-4.10663598e-11 -1.29426087e-02]
Перехват: 22.104442119889338


## Выводы
- Мы смогли проанализировать влияние ВВП и инфляции на объём торговли.
- Использовали линейную регрессию для прогнозирования.
- Интерактивные графики позволяют лучше понять данные.

## 1. Установка и импорт библиотек **Важно:** выполнение этой ячейки в среде Colab требует доступа в интернет и может занять несколько минут.

In [ ]:
# Установка необходимых пакетов (выполнять в Colab)
!pip install world_bank_data world_trade_data linearmodels wbgapi --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

import world_bank_data as wb  # доступ к данным Всемирного банка
import world_trade_data as wits  # доступ к WITS API (торговля)
import wbgapi as wbapi  # дополнительный доступ к World Bank API

from linearmodels import PanelOLS  # панельные регрессии

pd.set_option('display.max_columns', None)

In [ ]:
# Список ISO3-кодов frontier-стран по состоянию на 2025 год (по данным MSCI, FTSE и других источников)
frontier_iso3 = [
    'BDI',  # Бурунди
    'CPV',  # Кабо-Верде
    'KHM',  # Камбоджа
    'COG',  # Республика Конго
    'CIV',  # Кот-д'Ивуар
    'GNQ',  # Экваториальная Гвинея
    'GAB',  # Габон
    'GMB',  # Гамбия
    'GHA',  # Гана
    'KAZ',  # Казахстан
    'KGZ',  # Киргизия
    'LBY',  # Ливия
    'LKA',  # Шри-Ланка
    'MDV',  # Мальдивы
    'MLI',  # Мали
    'MNG',  # Монголия
    'MOZ',  # Мозамбик
    'MMR',  # Мьянма
    'NIC',  # Никарагуа
    'NER',  # Нигер
    'NGA',  # Нигерия
    'PAK',  # Пакистан
    'PAN',  # Панама
    'PNG',  # Папуа — Новая Гвинея
    'PRY',  # Парагвай
    'RWA',  # Руанда
    'SEN',  # Сенегал
    'SLE',  # Сьерра-Леоне
    'TJK',  # Таджикистан
    'TZA',  # Танзания
    'THA',  # Таиланд (частично frontier по некоторым классификациям)
    'TGO',  # Того
    'UKR',  # Украина
    'UZB',  # Узбекистан
    'VNM',  # Вьетнам
    'ZWE',  # Зимбабве
]

# Дополнительно: список стран, которые ранее были в статусе frontier, но были повышены до emerging
former_frontier_iso3 = [
    'BGR',  # Болгария
    'HRV',  # Хорватия
    'JOR',  # Иордания
    'KWT',  # Кувейт
    'ROU',  # Румыния
    'BLR',  # Беларусь
    'EST',  # Эстония
    'LVA',  # Латвия
    'LTU',  # Литва
    'CHN',  # Китай
    'IND',  # Индия
    'BRA',  # Бразилия
    'RUS',  # Россия (ранее была в emerging, сейчас часто рассматривается как frontier или standalone market)
]

print(f"Текущие frontier-страны ({len(frontier_iso3)}): {frontier_iso3}")
print(f"Бывшие frontier-страны ({len(former_frontier_iso3)}): {former_frontier_iso3}")

Текущие frontier-страны (36): ['BDI', 'CPV', 'KHM', 'COG', 'CIV', 'GNQ', 'GAB', 'GMB', 'GHA', 'KAZ', 'KGZ', 'LBY', 'LKA', 'MDV', 'MLI', 'MNG', 'MOZ', 'MMR', 'NIC', 'NER', 'NGA', 'PAK', 'PAN', 'PNG', 'PRY', 'RWA', 'SEN', 'SLE', 'TJK', 'TZA', 'THA', 'TGO', 'UKR', 'UZB', 'VNM', 'ZWE']
Бывшие frontier-страны (13): ['BGR', 'HRV', 'JOR', 'KWT', 'ROU', 'BLR', 'EST', 'LVA', 'LTU', 'CHN', 'IND', 'BRA', 'RUS']


In [ ]:
# Список ISO3-кодов frontier-стран по состоянию на 2025 год (по данным MSCI, FTSE и других источников)
frontier_iso3 = [
    'BDI',  # Бурунди
    'CPV',  # Кабо-Верде
    'KHM',  # Камбоджа
    'COG',  # Республика Конго
    'CIV',  # Кот-д'Ивуар
    'GNQ',  # Экваториальная Гвинея
    'GAB',  # Габон
    'GMB',  # Гамбия
    'GHA',  # Гана
    'KAZ',  # Казахстан
    'KGZ',  # Киргизия
    'LBY',  # Ливия
    'LKA',  # Шри-Ланка
    'MDV',  # Мальдивы
    'MLI',  # Мали
    'MNG',  # Монголия
    'MOZ',  # Мозамбик
    'MMR',  # Мьянма
    'NIC',  # Никарагуа
    'NER',  # Нигер
    'NGA',  # Нигерия
    'PAK',  # Пакистан
    'PAN',  # Панама
    'PNG',  # Папуа — Новая Гвинея
    'PRY',  # Парагвай
    'RWA',  # Руанда
    'SEN',  # Сенегал
    'SLE',  # Сьерра-Леоне
    'TJK',  # Таджикистан
    'TZA',  # Танзания
    'TGO',  # Того
    'UKR',  # Украина
    'UZB',  # Узбекистан
    'ZWE',  # Зимбабве
]

# Дополнительно: список стран, которые ранее были в статусе frontier, но были повышены до emerging
former_frontier_iso3 = [
    'BGR',  # Болгария
    'HRV',  # Хорватия
    'JOR',  # Иордания
    'KWT',  # Кувейт
    'ROU',  # Румыния
    'BLR',  # Беларусь
    'EST',  # Эстония
    'LVA',  # Латвия
    'LTU',  # Литва
    'CHN',  # Китай
    'IND',  # Индия
    'BRA',  # Бразилия
    'RUS',  # Россия (ранее была в emerging, сейчас часто рассматривается как frontier или standalone market)
    'THA',  # Таиланд
    'VNM',  # Вьетнам
]

print(f"Текущие frontier-страны ({len(frontier_iso3)}): {frontier_iso3}")
print(f"Бывшие frontier-страны ({len(former_frontier_iso3)}): {former_frontier_iso3}")

Текущие frontier-страны (34): ['BDI', 'CPV', 'KHM', 'COG', 'CIV', 'GNQ', 'GAB', 'GMB', 'GHA', 'KAZ', 'KGZ', 'LBY', 'LKA', 'MDV', 'MLI', 'MNG', 'MOZ', 'MMR', 'NIC', 'NER', 'NGA', 'PAK', 'PAN', 'PNG', 'PRY', 'RWA', 'SEN', 'SLE', 'TJK', 'TZA', 'TGO', 'UKR', 'UZB', 'ZWE']
Бывшие frontier-страны (15): ['BGR', 'HRV', 'JOR', 'KWT', 'ROU', 'BLR', 'EST', 'LVA', 'LTU', 'CHN', 'IND', 'BRA', 'RUS', 'THA', 'VNM']


In [ ]:
# Примерный список ISO3-кодов frontier-стран (ЗАМЕНИТЕ на актуальный список по данным Всемирного банка / MSCI / FTSE)
frontier_iso3 = [
    'VNM',  # Вьетнам
    'RWA',  # Руанда
    'NGA',  # Нигерия
    'KAZ',  # Казахстан
    'KEN',  # Кения
    'GHA',  # Гана
    'LKA',  # Шри-Ланка
    'BGR',  # Болгария (пример страны, вышедшей из frontier-статуса)
    'ROU',  # Румыния
]
len(frontier_iso3), frontier_iso3

(9, ['VNM', 'RWA', 'NGA', 'KAZ', 'KEN', 'GHA', 'LKA', 'BGR', 'ROU'])

## 2. Список frontier-стран. В реальном исследовании список frontier-стран формируется на основе приложений к докладу Всемирного банка "Frontier Market Economies" или классификаций MSCI/FTSE. Ниже приведён **пример** списка ISO3-кодов; его необходимо скорректировать под актуальный перечень стран.

In [ ]:
# Примерный список ISO3-кодов frontier-стран (ЗАМЕНИТЕ на актуальный список по данным Всемирного банка / MSCI / FTSE)
frontier_iso3 = [
    'VNM',  # Вьетнам
    'RWA',  # Руанда
    'NGA',  # Нигерия
    'KAZ',  # Казахстан
    'KEN',  # Кения
    'GHA',  # Гана
    'LKA',  # Шри-Ланка
    'BGR',  # Болгария (пример страны, вышедшей из frontier-статуса)
    'ROU',  # Румыния
]

print(len(frontier_iso3), frontier_iso3)

9 ['VNM', 'RWA', 'NGA', 'KAZ', 'KEN', 'GHA', 'LKA', 'BGR', 'ROU']


## 3. Макроэкономические индикаторы frontier-стран (World Bank WDI)\n\nВ этом разделе:\n- загружаем ключевые индикаторы Всемирного банка для frontier-стран;\n- рассчитываем среднюю торговую открытость группы (Trade (% of GDP)).

In [ ]:
# 3.1. Информация о странах
countries = wb.get_countries() # Получаем данные по странам от Всемирного банка
countries = countries.reset_index() # Сбрасываем индекс, чтобы 'id' стал обычной колонкой
display(countries.head()) # Отображаем первые строки таблицы стран

# 3.2. Фильтрация по списку frontier-стран
frontier = countries[countries['id'].isin(frontier_iso3)].copy() # Фильтруем страны по списку frontier_iso3
print('Число отобранных frontier-стран:', len(frontier)) # Выводим количество отобранных стран
display(frontier[['id', 'name', 'region']].head()) # Отображаем первые строки отфильтрованных стран с их ID, именем и регионом

# 3.3. Загрузка индикаторов WDI
indicators = {
    'NY.GDP.MKTP.KD': 'gdp_const',             # ВВП (конст. цены 2015 г.)
    'NE.EXP.GNFS.ZS': 'exp_gdp',              # экспорт товаров и услуг (% ВВП)
    'NE.IMP.GNFS.ZS': 'imp_gdp',              # импорт товаров и услуг (% ВВП)
    'NE.TRD.GNFS.ZS': 'trade_gdp',            # торговля (% ВВП)
}

try:
    data = wb.get_series(
        list(indicators.keys()), # Список кодов индикаторов для загрузки
        id_or_value='id', # Указываем, что используем ID индикаторов
        country=','.join(frontier_iso3), # Преобразуем список ISO-кодов стран в строку, разделенную запятыми
        date='2000:2024', # Указываем временной диапазон данных (с 2000 по 2024 год)
    )

    data = data.unstack(level=0).rename(columns=indicators).reset_index() # Разворачиваем данные и переименовываем колонки
    data.rename(columns={'country': 'iso3', 'time': 'year'}, inplace=True) # Переименовываем колонки 'country' и 'time'
    data['year'] = data['year'].astype(int) # Преобразуем колонку 'year' в целочисленный тип

    print(data.head()) # Отображаем первые строки загруженных данных

    # 3.4. Средняя торговая открытость frontier-стран по годам
    trade_openness = (
        data.groupby('year')['trade_gdp'] # Группируем данные по годам и выбираем 'trade_gdp'
        .mean() # Вычисляем среднее значение 'trade_gdp' для каждого года
        .reset_index() # Сбрасываем индекс, чтобы 'year' стал обычной колонкой
        .rename(columns={'trade_gdp': 'trade_gdp_frontier_avg'}) # Переименовываем колонку 'trade_gdp' в 'trade_gdp_frontier_avg'
    )

    display(trade_openness.tail()) # Отображаем последние строки таблицы средней торговой открытости
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    # Инициализируем пустой DataFrame, если загрузка не удалась
    data = pd.DataFrame()
    trade_openness = pd.DataFrame()

,id,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,,High income,Not classified,Oranjestad,-70.0167,12.5167
1,AFE,ZH,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates,,NaN,NaN
2,AFG,AF,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan","Middle East, North Africa, Afghanistan & Pakis...",Low income,IDA,Kabul,69.1761,34.5228
3,AFR,A9,Africa,Aggregates,,Aggregates,Aggregates,,NaN,NaN
4,AFW,ZI,Africa Western and Central,Aggregates,,Aggregates,Aggregates,,NaN,NaN


Число отобранных frontier-стран: 9


,id,name,region
24,BGR,Bulgaria,Europe & Central Asia
105,GHA,Ghana,Sub-Saharan Africa
143,KAZ,Kazakhstan,Europe & Central Asia
144,KEN,Kenya,Sub-Saharan Africa
161,LKA,Sri Lanka,South Asia


Ошибка при загрузке данных: sequence item 4: expected str instance, list found


## 4. Торговые потоки Россия — frontier-страны (WITS / UN Comtrade). В этом разделе формируется временной ряд торгового оборота России с frontier-странами.
**Примечание:** WITS и/или UN Comtrade могут требовать регистрации и API-ключа. Убедитесь, что у вас есть необходимые права доступа.

In [ ]:
import pandas as pd

# 4.1. Параметры запроса
reporter = 'russian federation'  # страна-репортёр
partners = [code.lower() for code in frontier_iso3]
years = list(range(2000, 2024))

records = []

try:
    for year in years:
        # Экспорт России в frontier-страны
        df_exp = wits.get_indicator(
            'XPRT-TRD-VL',
            reporter=reporter,
            partner=partners,
            year=year,
            name_or_id='id',
        )

        # Импорт России из frontier-стран
        df_imp = wits.get_indicator(
            'MPRT-TRD-VL',
            reporter=reporter,
            partner=partners,
            year=year,
            name_or_id='id',
        )

        df_merged = (
            df_exp[['partnercode', 'value']]
            .rename(columns={'value': 'export_usd'})
            .merge(
                df_imp[['partnercode', 'value']].rename(columns={'value': 'import_usd'}),
                on='partnercode',
                how='outer',
            )
        )
        df_merged['year'] = year
        records.append(df_merged)

    trade_rf_frontier = pd.concat(records, ignore_index=True)
    trade_rf_frontier.rename(columns={'partnercode': 'iso3'}, inplace=True)

    trade_rf_frontier_total = (
        trade_rf_frontier.groupby('year')[['export_usd', 'import_usd']]
        .sum()
        .reset_index()
    )
    trade_rf_frontier_total['turnover_usd'] = (
        trade_rf_frontier_total['export_usd'] + trade_rf_frontier_total['import_usd']
    )

    print(trade_rf_frontier_total.tail())

except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    # Инициализируем пустой DataFrame, если загрузка не удалась
    trade_rf_frontier_total = pd.DataFrame()

Ошибка при загрузке данных: argument of type 'int' is not iterable


## 5. Доля frontier-рынков в общем торговом обороте России. Для расчёта доли требуется ряд общего внешнеторгового оборота России по годам. Источник может быть: - WITS (тот же API); - официальная статистика Росстата / ФТС РФ (последующее ручное добавление в CSV).

In [ ]:
# 5.1. Загрузка или чтение общего оборота России
# Вариант 1: чтение из заранее подготовленного CSV-файла
# total_turnover_rf = pd.read_csv('russia_total_turnover.csv')
# Ожидаемая структура: year, turnover_usd_total

# Вариант 2: заглушка (пример структуры)
if not trade_rf_frontier_total.empty:
    total_turnover_rf = trade_rf_frontier_total[['year']].copy()
    total_turnover_rf['turnover_usd_total'] = total_turnover_rf['year'].apply(
        lambda y: 5e11 + (y - total_turnover_rf['year'].min()) * 1e10
    )  # ИСКЛЮЧИТЕЛЬНО ДЛЯ ДЕМОНСТРАЦИИ! Замените на реальные данные.

    # 5.2. Расчёт доли frontier-рынков в общем обороте
    df_share = trade_rf_frontier_total.merge(total_turnover_rf, on='year', how='inner')
    df_share['share_frontier_pct'] = (
        df_share['turnover_usd'] / df_share['turnover_usd_total'] * 100
    )

    df_share.tail()
else:
    print("Данные по торговле с frontier-странами отсутствуют. Невозможно рассчитать долю.")
    df_share = pd.DataFrame()

Данные по торговле с frontier-странами отсутствуют. Невозможно рассчитать долю.


## 6. Синтетический (учебный) панельный набор данных. Здесь создаётся синтетический датасет "год–страна–ВВП–экспорт–импорт". Он используется только для иллюстрации методов и **не заменяет реальные данные**.

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)
years_demo = np.arange(2010, 2025)
countries_demo = frontier_iso3

rows = []
for iso in countries_demo:
    base_gdp = np.random.uniform(5e9, 1e11)  # базовый ВВП
    last_gdp = base_gdp
    for year in years_demo:
        growth = np.random.normal(0.03, 0.02)  # средний рост 3% с шумом
        gdp = last_gdp * (1 + growth)
        export = gdp * np.random.uniform(0.15, 0.35)
        impt = gdp * np.random.uniform(0.15, 0.35)
        rows.append({
            'iso3': iso,
            'year': year,
            'gdp_const': gdp,
            'export_usd': export,
            'import_usd': impt,
        })
        last_gdp = gdp

demo = pd.DataFrame(rows)
demo['turnover_usd'] = demo['export_usd'] + demo['import_usd']
demo['trade_gdp'] = demo['turnover_usd'] / demo['gdp_const'] * 100

demo.head()

,iso3,year,gdp_const,export_usd,import_usd,turnover_usd,trade_gdp
0,VNM,2010,4.089632e+10,7.410368e+09,6.609529e+09,1.401990e+10,34.281563
1,VNM,2011,4.238405e+10,1.370002e+10,1.145314e+10,2.515316e+10,59.345823
2,VNM,2012,4.316317e+10,8.057739e+09,9.100887e+09,1.715863e+10,39.752935
3,VNM,2013,4.400470e+10,1.121906e+10,1.040223e+10,2.162128e+10,49.134029
4,VNM,2014,4.604339e+10,8.191063e+09,9.596775e+09,1.778784e+10,38.632770


## 7. Пример панельной регрессии (на основе данных) Ниже приведён пример оценки простой панельной регрессии зависимости логарифма торгового оборота от логарифмов ВВП и торговой открытости frontier-стран. Можно использовать **реальные данные**, полученные выше (объединить `trade_rf_frontier` и `data`).  Либо для отладки использовать синтетический датасет `demo`.

In [ ]:
from linearmodels import PanelOLS
import numpy as np
import pandas as pd

# 7.1. Объединение синтетических торговых данных и макроиндикаторов
panel_demo = demo.copy()
panel_demo['year'] = panel_demo['year'].astype(int)

# Для демонстрации считаем, что trade_gdp уже рассчитан в demo
panel_demo = panel_demo.set_index(['iso3', 'year'])

# 7.2. Подготовка переменных
y = np.log(panel_demo['turnover_usd'] + 1)
X = panel_demo[['gdp_const', 'trade_gdp']]
X = np.log(X + 1)

# 7.3. Оценка модели с фиксированными эффектами по странам и годам
model = PanelOLS(y, X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='clustered', cluster_entity=True)
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:           turnover_usd   R-squared:                        1.0000
Estimator:                   PanelOLS   R-squared (Between):              0.9613
No. Observations:                 135   R-squared (Within):               1.0000
Date:                Sat, Feb 07 2026   R-squared (Overall):              0.9613
Time:                        11:31:46   Log-likelihood                    874.27
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.087e+07
Entities:                           9   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                   F(2,110)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):          5.656e+06
                            

# Задача
Измените ячейку `2eaaec3c`, чтобы получить данные Всемирного банка, используя `wbgapi.data.fetch`, затем обработайте данные, переименовав столбцы ( `economyв iso3, timeв year`), преобразовав `year` в целочисленный формат, объединив коды индикаторов в столбцы и переименовав их в соответствии со `indicators` словарем, обеспечив при этом совместимость последующего расчета открытости торговли..

In [ ]:
import pandas as pd
import numpy as np

import world_bank_data as wb  # доступ к данным Всемирного банка
import world_trade_data as wits  # доступ к WITS API (торговля)
import wbgapi as wbapi  # дополнительный доступ к World Bank API

from linearmodels import PanelOLS  # панельные регрессии

pd.set_option('display.max_columns', None)

# 3.1. Информация о странах
countries = wb.get_countries() # Получаем данные по странам от Всемирного банка
countries = countries.reset_index() # Сбрасываем индекс, чтобы 'id' стал обычной колонкой
display(countries.head()) # Отображаем первые строки таблицы стран

# 3.2. Фильтрация по списку frontier-стран
frontier = countries[countries['id'].isin(frontier_iso3)].copy() # Фильтруем страны по списку frontier_iso3
print('Число отобранных frontier-стран:', len(frontier)) # Выводим количество отобранных стран
display(frontier[['id', 'name', 'region']].head()) # Отображаем первые строки отфильтрованных стран с их ID, именем и регионом

# 3.3. Загрузка индикаторов WDI
indicators = {
    'NY.GDP.MKTP.KD': 'gdp_const',             # ВВП (конст. цены 2015 г.)
    'NE.EXP.GNFS.ZS': 'exp_gdp',              # экспорт товаров и услуг (% ВВП)
    'NE.IMP.GNFS.ZS': 'imp_gdp',              # импорт товаров и услуг (% ВВП)
    'NE.TRD.GNFS.ZS': 'trade_gdp',            # торговля (% ВВП)
}

try:
    data_raw = wbapi.data.fetch(
        series=list(indicators.keys()), # Список кодов индикаторов для загрузки
        economy=frontier_iso3, # Список ISO-кодов стран
        time='2000:2024' # Указываем временной диапазон данных (с 2000 по 2024 год)
    )
    data = pd.DataFrame(data_raw)

    data = data.rename(columns={'economy': 'iso3', 'time': 'year', 'series': 'indicator_code'}) # Переименовываем колонки
    data['year'] = data['year'].astype(int) # Преобразуем колонку 'year' в целочисленный тип

    # Pivot the table to have indicators as columns
    data = data.pivot_table(index=['iso3', 'year'], columns='indicator_code', values='value').reset_index()
    data = data.rename(columns=indicators) # Rename indicator codes to friendly names

    print(data.head()) # Отображаем первые строки загруженных данных

    # 3.4. Средняя торговая открытость frontier-стран по годам
    trade_openness = (
        data.groupby('year')['trade_gdp'] # Группируем данные по годам и выбираем 'trade_gdp'
        .mean() # Вычисляем среднее значение 'trade_gdp' для каждого года
        .reset_index() # Сбрасываем индекс, чтобы 'year' стал обычной колонкой
        .rename(columns={'trade_gdp': 'trade_gdp_frontier_avg'}) # Переименовываем колонку 'trade_gdp' в 'trade_gdp_frontier_avg'
    )

    display(trade_openness.tail()) # Отображаем последние строки таблицы средней торговой открытости

except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    # Инициализируем пустой DataFrame, если загрузка не удалась
    data = pd.DataFrame()
    trade_openness = pd.DataFrame()

,id,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,,High income,Not classified,Oranjestad,-70.0167,12.5167
1,AFE,ZH,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates,,NaN,NaN
2,AFG,AF,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan","Middle East, North Africa, Afghanistan & Pakis...",Low income,IDA,Kabul,69.1761,34.5228
3,AFR,A9,Africa,Aggregates,,Aggregates,Aggregates,,NaN,NaN
4,AFW,ZI,Africa Western and Central,Aggregates,,Aggregates,Aggregates,,NaN,NaN


Число отобранных frontier-стран: 9


,id,name,region
24,BGR,Bulgaria,Europe & Central Asia
105,GHA,Ghana,Sub-Saharan Africa
143,KAZ,Kazakhstan,Europe & Central Asia
144,KEN,Kenya,Sub-Saharan Africa
161,LKA,Sri Lanka,South Asia


Ошибка при загрузке данных: APIError: [404] Resource Not Found (https://api.worldbank.org/v2/en/sources/2/series/NY.GDP.MKTP.KD;NE.EXP.GNFS.ZS;NE.IMP.GNFS.ZS;NE.TRD.GNFS.ZS/country/VNM;RWA;NGA;KAZ;KEN;GHA;LKA;BGR;ROU/time/2000:2024?per_page=1000&page=1&format=json)


**Практическая часть / Practical Part:**

* На основе проведённого анализа можно предложить следующие направления для внедрения в России:
* Разработка цифровой платформы для мониторинга экономических показателей стран-партнёров, интегрированной с API Всемирного банка.
* Автоматизация прогнозирования объёмов внешней торговли с использованием машинного обучения.
* Фокус на страны с растущей экономикой и стабильной политической ситуацией для расширения экспортных возможностей.

Заключение

Frontier‑рынки, согласно последнему выпуску Global Economic Prospects, обладают одновременно значительным демографическим потенциалом, относительным институциональным преимуществом и уязвимостью к внешним шокам, связанным с долговыми нагрузками и волатильностью потоков капитала. Для России, находящейся в процессе структурной перенастройки внешней торговли, эти экономики становятся важным сегментом партнёрской базы, особенно с учётом диверсификации от традиционных развитых рынков.[3][7][4]

Использование открытых данных Всемирного банка, WITS, UN Comtrade в сочетании с инструментами Python 3 (библиотеки world_bank_data, world_trade_data, специализированные скрипты интеграции) позволяет построить системный и воспроизводимый анализ влияния frontier‑рынков на торговый оборот России. Представленные в статье примеры кода демонстрируют основные этапы такого анализа: идентификацию множества frontier‑стран, загрузку панельных макро‑ и торговых данных, построение агрегированных показателей и простейших эконометрических моделей.

Развитие данного направления предполагает углубление анализа до уровня товарных групп (HS/SITC), учёт транспортно‑логистических ограничений, а также совместное использование макроэкономических сценариев (Всемирный банк, МВФ) и отраслевых прогнозов. На этой основе могут быть сформированы более точные рекомендательные модели для российских экспортёров, импортеров и государственных органов, направленные на минимизацию рисков и максимизацию выгод сотрудничества с frontier‑экономиками.


**Библиографический список / Bibliography**:
* Всемирный банк. Глобальные экономические перспективы. – М., 2026. – URL: https://www.vsemirnyjbank.org/ru/publication/global-economic-prospects (дата обращения: 05.09.2025).
* World Bank Open Data. International Trade Statistics. – URL: https://data.worldbank.org/
* McKinsey Global Institute. Global Economic Prospects and the Developing World. – Washington, D.C., 2025.
* GitHub Repository: Economic Indicators Analysis. – URL: https://github.com/user/economic-indicators-analysis
* Kaggle Dataset: Russia’s International Trade Data. – URL: https://www.kaggle.com/datasets/russia-trade-data
* Pandas Documentation. Data Manipulation in Python. – URL: https://pandas.pydata.org/
* Seaborn Library. Statistical Data Visualization. – URL: https://seaborn.pydata.org/

Библиографический список (по ГОСТ Р 7.0.5‑2008, примерное оформление)
Frontier Market Economies:
1. Promise, Performance, and Prospects // Global Economic Prospects. January 2026. – Washington, DC : World Bank, 2026. – 200 p. – URL: https://openknowledge.worldbank.org/bitstreams/021dbb36-16b7-4d2e-bcd5-5c05f2643591/download (дата обращения: 07.02.2026).[3]
2. Frontier markets – a subset of emerging market and developing economies characterized by meaningful but limited access to international financial markets // World Bank. Frontier Market Economies. – Washington, DC : World Bank, 2026. – URL: https://www.worldbank.org/en/research/publication/frontier-markets (дата обращения: 07.02.2026).[1]
3. ‘Frontier Market’ Economies Haven’t Lived Up to Potential – New World Bank Study Finds. – Press release. – Washington, DC : World Bank, 20 Jan. 2026. – URL: https://www.worldbank.org/en/news/press-release/2026/01/20/frontier-markets-press-release (дата обращения: 07.02.2026).[2]
4. Global Economic Prospects. – Official website. – Washington, DC : World Bank, 2026. – URL: https://www.worldbank.org/en/publication/global-economic-prospects (дата обращения: 07.02.2026).[31]
5. Frontier market // Wikipedia, the free encyclopedia. – 2025. – URL: https://en.wikipedia.org/wiki/Frontier_market (дата обращения: 07.02.2026).[8]
6. List of Frontier Markets & Classifications // Frontier Market News. – 2023. – URL: https://frontiermarketnews.org/list-of-frontier-markets/ (дата обращения: 07.02.2026).[9]
4. Frontier Market // ScienceDirect Topics. – Elsevier, 2015. – URL: https://www.sciencedirect.com/topics/social-sciences/frontier-market (дата обращения: 07.02.2026).[10]
Emerging countries have replaced most of Russia’s lost trade with advanced economies // Bruegel. – 04.01.2026. – URL: https://www.bruegel.org/analysis/emerging-countries-have-replaced-most-russias-lost-trade-advanced-economies (дата обращения: 07.02.2026).[4]
8. Russian Federation Trade Statistics // World Integrated Trade Solution (WITS). – Washington, DC : World Bank, 2024. – URL: https://wits.worldbank.org/CountryProfile/en/RUS (дата обращения: 07.02.2026).[5]
9. RUSSIA Trade | WITS Data // World Bank. – 2024. – URL: https://wits.worldbank.org/countrysnapshot/Russia (дата обращения: 07.02.2026).[24]
10. Trade – Data. World Bank Open Data. – Washington, DC : World Bank, 2025. – URL: https://data.worldbank.org/topic/21 (дата обращения: 07.02.2026).[6]
11. Indicators – World Bank Open Data. – Washington, DC : World Bank, 2025. – URL: https://data.worldbank.org/indicator (дата обращения: 07.02.2026).[27]
12. World Bank Open Data. – Washington, DC : World Bank, 2026. – URL: https://data.worldbank.org (дата обращения: 07.02.2026).[28]
13. Mwouts. world_bank_data : The World Bank Data in Python [Электронный ресурс]. – GitHub repository, 2019. – URL: https://github.com/mwouts/world_bank_data (дата обращения: 07.02.2026).[15]
14. Mwouts. world_trade_data : World Trade Statistics (WITS) API in Python [Электронный ресурс]. – GitHub repository, 2019. – URL: https://github.com/mwouts/world_trade_data (дата обращения: 07.02.2026).[17][16]
15. World Bank Data // Kaggle dataset / R. Damarla. – 2020. – URL: https://www.kaggle.com/datasets/rishidamarla/world-bank-data (дата обращения: 07.02.2026).[20]
16. Tripathy S. GDP‑Per‑Capita_Gov‑Expenditure_Trade – Hugging Face Datasets [Электронный ресурс]. – 2025. – URL: https://huggingface.co/datasets/tripathyShaswata/GDP-Per-Capita_Gov-Expenditure_Trade (дата обращения: 07.02.2026).[21]
17. Pattawee P. UN Comtrade & World Bank API: Python‑based automation tool for downloading, merging, and summarizing international trade and macroeconomic data [Электронный ресурс]. – 2024. – URL: https://www.pattawee-pp.com/trade-talk/2025/trade-research-api/ (дата обращения: 07.02.2026).[18]
18. Joshi S. World Bank Data Analysis (EDA) [Электронный ресурс]. – GitHub, 2020. – URL: https://github.com/shwetajoshi601/world-bank-data-analysis/blob/master/world-bank-data-eda.ipynb (дата обращения: 07.02.2026).[19]
19. Олейнов А.Г. Теории международной торговли и внешнеторгового регулирования. Часть I // Мировая экономика и международные отношения. – М. : МГИМО, 2010. – URL: https://mirec.mgimo.ru/upload/ckeditor/files/olejnov-teorii-mezhdunarodnoj-torgovli-i-vneshnetorgovogo-regulirovaniya-chast-i.pdf (дата обращения: 07.02.2026).[14]

Список рекомендованной литературы по мировой экономике и конъюнктуре товарных рынков [Электронный ресурс]. – URL: http://edoc.bseu.by:8080/bitstream/edoc/4373/19/Spisok rekomend. lit-ry.doc (дата обращения: 07.02.2026).[12]

Международная торговля : расширенный список литературы [Электронный ресурс]. – URL: http://list-of-lit.ru/narod/mejdunarodnaya-torgovlya.htm (дата обращения: 07.02.2026).[13]